AUTENTICAÇÃO E BIBLIOTECAS

In [ ]:
import pandas as pd
import json
import requests

import google.auth
from google.oauth2 import service_account
from google.auth.transport.requests import Request
from google.cloud import storage
from google.cloud import bigquery
from google.auth import default

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
SCOPES = ['https://www.googleapis.com/auth/cloud-platform']

# Get credentials from the environment
credentials__, project_id__ = default(scopes=SCOPES)

# Refresh the credentials if necessary
if not credentials__.valid or credentials__.expired:
    credentials__.refresh(Request())

In [ ]:
PROJECT_ID = project_id__

RELAÇÃO DE TAGS, USERS, DATASETS

In [ ]:
#De para de datasets e usuários
#De para de datasets e tags
#Relação das duas bases

In [ ]:
users_datasets_path = r'C:\de_para_access_group_dataset.csv' # Update this path to the correct location of your CSV file
users_datasets = pd.read_csv(users_datasets_path, delimiter = ';')
users_datasets

In [ ]:
datasets_tags_path = r'C:\de_para_dataset_tag.csv' # Update this path to the correct location of your CSV file
datasets_tags = pd.read_csv(datasets_tags_path, delimiter = ';')
datasets_tags

In [ ]:
users_datasets_relationship = users_datasets.merge(datasets_tags, on='dataset', how='left')
users_datasets_relationship['tag_key'] = PROJECT_ID + '/' + users_datasets_relationship['tag_key']

TAGS | IAM

In [ ]:
#Ler tags do IAM
#De-para com tags listadas
#Se a tag não existe, criar

In [ ]:
# Prepare the URL for the GET request to list all tag keys
url = f"https://cloudresourcemanager.googleapis.com/v3/tagKeys?parent=projects/{PROJECT_ID}"

# Prepare the headers with the obtained access token
headers = {
    'Authorization': f"Bearer {credentials__.token}",
    'Content-Type': 'application/json'
}

# Perform the GET request
response = requests.get(url, headers=headers)

# Check the response status and print the result
if response.status_code == 200:
    print("Success!")
else:
    print("Failed:", response.status_code, response.text)

iam_tags = [tag_key['namespacedName'] for tag_key in response.json()['tagKeys']]

In [ ]:
def post_create_tag_key(project, tag_key_name, description = None):

    # Prepare the URL for the POST request
    url = "https://cloudresourcemanager.googleapis.com/v3/tagKeys/"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Define the JSON payload for the POST request
    payload = {
        "parent": f"projects/{project}",  # Replace RESOURCE_ID with your actual resource ID
        "shortName": f"{tag_key_name}",  # Replace SHORT_NAME with your actual short name
        "description": f"{description}"  # Replace DESCRIPTION with your actual description
    }

    # Convert the payload dictionary to a JSON formatted string
    json_data = json.dumps(payload)

    # Perform the POST request
    response = requests.post(url, headers=headers, data=json_data)

    # Check the response status and print the result
    if response.status_code == 200:
        return print("Success! Tag Created.")
    else:
        return print("Failed:", response.status_code, response.text)

In [ ]:
#Comparar key tags do dataset versus tags do IAM
for tag_key in list(users_datasets_relationship['tag_key'].unique()):

    if tag_key in iam_tags:
        print(f'Tag Key {tag_key} already exists')
    else:
        print(f'Tag Key {tag_key} does not exist')
        post_create_tag_key(tag_key.split('/')[0], tag_key.split('/')[1], f'TESTE TAG {tag_key}')

In [ ]:
# Prepare the URL for the GET request to list all tag keys
url = f"https://cloudresourcemanager.googleapis.com/v3/tagKeys?parent=projects/{PROJECT_ID}"

# Prepare the headers with the obtained access token
headers = {
    'Authorization': f"Bearer {credentials__.token}",
    'Content-Type': 'application/json'
}

# Perform the GET request
response = requests.get(url, headers=headers)

# Check the response status and print the result
if response.status_code == 200:
    print("Success!")
else:
    print("Failed:", response.status_code, response.text)

tag_key_shortname = [tag_key['namespacedName'] for tag_key in response.json()['tagKeys']]
tag_key_name = [tag_key['name'] for tag_key in response.json()['tagKeys']]

iam_tags = dict(zip(tag_key_shortname, tag_key_name))

iam_tags

In [ ]:
#Ler todas as tag values de uma tag key do IAM
tag_key_name = []
tag_key_shortname = []
tag_values_name = []
tag_values_shortname = []

for key, item in iam_tags.items():

    # Prepare the URL for the GET request to list all tag keys
    url = f"https://cloudresourcemanager.googleapis.com/v3/tagValues?parent={item}"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Perform the GET request
    response = requests.get(url, headers=headers)

    # Check the response status and print the result
    if response.status_code == 200 and response.json() == {}:
        tag_key_name.append(item)
        tag_key_shortname.append(key)
        tag_values_name.append([])
        tag_values_shortname.append([])

    elif response.status_code == 200:
        tag_key_name.append(item)
        tag_key_shortname.append(key)
        tag_values_name.append([tagvalue['name'] for tagvalue in response.json()['tagValues']])
        tag_values_shortname.append([tagvalue['shortName'] for tagvalue in response.json()['tagValues']])
        
    else:
        print("Tag key {key} resquest failed:", response.status_code, response.text)

tag_keys_and_values = pd.DataFrame(data = {
                        'tag_key_shortname': tag_key_shortname
                        , 'tag_key_name': tag_key_name
                        , 'tag_values_shortname': tag_values_shortname
                        , 'tag_values_name': tag_values_name
                    })

tag_keys_and_values = tag_keys_and_values.explode(column = ['tag_values_name', 'tag_values_shortname'])


In [ ]:
df1 = users_datasets_relationship
df2 = tag_keys_and_values

result = df1.merge(df2, 
                  left_on=['tag_key']
                  , right_on=['tag_key_shortname']
                  , how='outer'
                )

tag_values_to_create = result[(result.dataset.isna() == False) & (result.tag_values_name.isna() == True)][['tag_key', 'tag_value', 'tag_key_name']].drop_duplicates()

In [ ]:
def post_create_tag_value(tag_key_name, tag_value_shortname, description = None):

    # Prepare the URL for the POST request
    url = "https://cloudresourcemanager.googleapis.com/v3/tagValues/"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Define the JSON payload for the POST request
    payload = {
        "parent": f"{tag_key_name}",
        "shortName": f"{tag_value_shortname}",
        "description": f"{description}"
    }

    # Convert the payload dictionary to a JSON formatted string
    json_data = json.dumps(payload)

    # Perform the POST request
    response = requests.post(url, headers=headers, data=json_data)

    # Check the response status and print the result
    if response.status_code == 200:
        return print("Success! Tag Value Created.")
    else:
        return print("Failed:", response.status_code, response.text)

In [ ]:
for index in range(len(tag_values_to_create)):

    post_create_tag_value(
        tag_values_to_create.loc[index, 'tag_key_name']
        , tag_values_to_create.loc[index, 'tag_value']
        , f'TESTE TAG {tag_values_to_create.loc[index, 'tag_value']}')

In [ ]:
tag_key_name = []
tag_key_shortname = []
tag_values_name = []
tag_values_shortname = []

for key, item in iam_tags.items():

    # Prepare the URL for the GET request to list all tag keys
    url = f"https://cloudresourcemanager.googleapis.com/v3/tagValues?parent={item}"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Perform the GET request
    response = requests.get(url, headers=headers)

    # Check the response status and print the result
    if response.status_code == 200 and response.json() == {}:
        tag_key_name.append(item)
        tag_key_shortname.append(key)
        tag_values_name.append([])
        tag_values_shortname.append([])

    elif response.status_code == 200:
        tag_key_name.append(item)
        tag_key_shortname.append(key)
        tag_values_name.append([tagvalue['name'] for tagvalue in response.json()['tagValues']])
        tag_values_shortname.append([tagvalue['shortName'] for tagvalue in response.json()['tagValues']])
        
    else:
        print("Tag key {key} resquest failed:", response.status_code, response.text)

tag_keys_and_values = pd.DataFrame(data = {
                        'tag_key_shortname': tag_key_shortname
                        , 'tag_key_name': tag_key_name
                        , 'tag_values_shortname': tag_values_shortname
                        , 'tag_values_name': tag_values_name
                    })

tag_keys_and_values = tag_keys_and_values.explode(column = ['tag_values_name', 'tag_values_shortname'])

df1 = users_datasets_relationship
df2 = tag_keys_and_values

result = df1.merge(df2, 
                  left_on=['tag_key']
                  , right_on=['tag_key_shortname']
                  , how='outer'
                )

tag_values_created = result[(result.dataset.isna() == False) & (result.tag_values_name.isna() == False)]

DATASETS | LEITURA DE TAGS E TAGUEAMENTO

In [ ]:
map_dataset_tags = tag_values_created.drop(columns = ['user_type', 'email']).drop_duplicates()

In [ ]:
# Initialize a BigQuery client
bq_client = bigquery.Client()

REGION = 'region-xxx'

In [ ]:
resources = []
dataset_has_tag_value = []

for dataset_ in map_dataset_tags['dataset']:

    # Specify your dataset ID (including your project ID and dataset name)
    dataset_id = f'{PROJECT_ID}.{dataset_}'

    # Fetch the dataset
    dataset = bq_client.get_dataset(dataset_id)

    DATASET_PATH = dataset.path

    # Prepare the URL for the GET request to list all tag keys
    RESOURCE_NAME = f"//bigquery.googleapis.com{DATASET_PATH}"
    url = f"https://{REGION}-cloudresourcemanager.googleapis.com/v3/tagBindings?parent={RESOURCE_NAME}"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Perform the GET request
    #response = requests.get(url, headers=headers)
    response = requests.get(url, headers=headers, verify=False)

    # Check the response status and print the result
    if response.status_code == 200 and response.json() == {}: #SE RETORNAR RECURSO (DATASET) SEM TAG
        print("Success!")        
        resources.append(RESOURCE_NAME)
        dataset_has_tag_value.append([])   
    elif response.status_code == 200: #RETORNA RECURSO (DATASET) COM TAG
        print("Success!")        
        resources.append(RESOURCE_NAME)
        dataset_has_tag_value.append([tagBindings['tagValue'] for tagBindings in response.json()['tagBindings']])     
    else:
        print("Failed:", response.status_code, response.text)

In [ ]:
datasets_tagged = pd.DataFrame(data = {
                        'dataset': [resource.split('/')[-1] for resource in resources]
                        , 'dataset_has_tag_value': dataset_has_tag_value
                    })

datasets_tagged = datasets_tagged.explode(column = ['dataset_has_tag_value'])

In [ ]:
datasets_tagged

In [ ]:
df1 = tag_values_created
df2 = datasets_tagged

mapped_tagged_datasets = df1.merge(df2, 
                          left_on=['dataset','tag_values_name']
                          , right_on=['dataset', 'dataset_has_tag_value']
                          , how='outer'
                        )[result['email'].isna() == False] #excluir datasets não mapeados no de-para

In [ ]:
mapped_tagged_datasets

In [ ]:
to_tag = mapped_tagged_datasets[mapped_tagged_datasets['dataset_has_tag_value'].isna() == True].drop(columns = ['user_type', 'email']).drop_duplicates()

In [ ]:
### TAGUEAMENTO DO DATASET
def post_tag_dataset(DATASET, TAG_KEY, TAG_VALUE):

    # Specify your dataset ID (including your project ID and dataset name)
    dataset_id = f'{PROJECT_ID}.{DATASET}'

    # Fetch the dataset
    dataset = bq_client.get_dataset(dataset_id)

    DATASET_PATH = dataset.path

    # Prepare the URL for the POST request
    url = f"https://{REGION}-cloudresourcemanager.googleapis.com/v3/tagBindings"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Define the JSON payload for the POST request
    payload = {
        "parent": f"//bigquery.googleapis.com{DATASET_PATH}",
        "tagValueNamespacedName": f"{TAG_KEY}/{TAG_VALUE}"
    }

    # Convert the payload dictionary to a JSON formatted string
    json_data = json.dumps(payload)

    # Perform the POST request
    #response = requests.post(url, headers=headers, data=json_data)
    response = requests.post(url, headers=headers, data=json_data, verify = False)

    # Check the response status and print the result
    if response.status_code == 409:
        print("Already tagged", response.json())
    elif response.status_code == 200:
        print("Success:", response.json())
    else:
        print("Failed:", response.status_code, response.text)

In [ ]:
for index in range(len(to_tag)):

    post_tag_dataset(
        to_tag.loc[index, 'dataset']
        , to_tag.loc[index, 'tag_key_shortname']
        , to_tag.loc[index, 'tag_values_shortname']
    )

In [ ]:
###Fazer a listagem e de-para novamente
resources = []
dataset_has_tag_value = []

for dataset_ in mapped_tagged_datasets['dataset'].unique():

    # Specify your dataset ID (including your project ID and dataset name)
    dataset_id = f'{PROJECT_ID}.{dataset_}'

    # Fetch the dataset
    dataset = bq_client.get_dataset(dataset_id)

    DATASET_PATH = dataset.path

    # Prepare the URL for the GET request to list all tag keys
    RESOURCE_NAME = f"//bigquery.googleapis.com{DATASET_PATH}"
    url = f"https://{REGION}-cloudresourcemanager.googleapis.com/v3/tagBindings?parent={RESOURCE_NAME}"

    # Prepare the headers with the obtained access token
    headers = {
        'Authorization': f"Bearer {credentials__.token}",
        'Content-Type': 'application/json'
    }

    # Perform the GET request
    #response = requests.get(url, headers=headers)
    response = requests.get(url, headers=headers, verify=False)

    # Check the response status and print the result
    if response.status_code == 200 and response.json() == {}:
        print("Success!")        
        resources.append(RESOURCE_NAME)
        dataset_has_tag_value.append([])   
    elif response.status_code == 200:
        print("Success!")        
        resources.append(RESOURCE_NAME)
        dataset_has_tag_value.append([tagBindings['tagValue'] for tagBindings in response.json()['tagBindings']])     
    else:
        print("Failed:", response.status_code, response.text)

In [ ]:
datasets_tagged = pd.DataFrame(data = {
                        'dataset': [resource.split('/')[-1] for resource in resources]
                        , 'dataset_has_tag_value': dataset_has_tag_value
                    })

datasets_tagged = datasets_tagged.explode(column = ['dataset_has_tag_value']).drop_duplicates()

In [ ]:
datasets_tagged

In [ ]:
tag_values_created

In [ ]:
df1 = tag_values_created
df2 = datasets_tagged

after_tagging_datasets = df1.merge(df2, 
                          left_on=['dataset','tag_values_name']
                          , right_on=['dataset', 'dataset_has_tag_value']
                          , how='outer'
                        )[result['email'].isna() == False] #excluir datasets não mapeados no de-para

In [ ]:
after_tagging_datasets

In [ ]:
apply_conditional_iam = after_tagging_datasets[['user_type','email', 'tag_key_shortname', 'tag_values_shortname']]

In [ ]:
apply_conditional_iam

APLICAÇÃO DE TAGS EM CONDICIONAIS DO IAM - ROLE BIGQUERY DATA VIEWER

In [ ]:
### LISTAR POLICIES

# Prepare the URL for the GET request to list all tag keys
url = f'https://cloudresourcemanager.googleapis.com/v1/projects/{PROJECT_ID}:getIamPolicy'

# Prepare the headers with the obtained access token
headers = {
    'Authorization': f"Bearer {credentials__.token}",
    'Content-Type': 'application/json'
}

# POST request data
data = '{ \"options\": {\"requestedPolicyVersion\": 3}}'

# Make the API request
response = requests.post(url, headers=headers, data=data)

In [ ]:
response.json()

In [ ]:
json_data = response.json()

In [ ]:
for binding in json_data['bindings']:
    if binding['role'] == 'roles/bigquery.dataViewer' and 'condition' not in binding:
        print(binding)
    elif binding['role'] == 'roles/bigquery.dataViewer': 
        print('\n')
        print(binding)
    else:
        pass

In [ ]:
apply_conditional_iam['user_and_email'] = apply_conditional_iam.user_type + ':' + apply_conditional_iam.email

In [ ]:
apply_conditional_iam['match_tag'] = 'resource.matchTag("' + apply_conditional_iam.tag_key_shortname + '", "' + apply_conditional_iam.tag_values_shortname+ '")' 

In [ ]:
condition_expression = []

for user_ in apply_conditional_iam.user_and_email.unique():

    if len(apply_conditional_iam[['user_and_email', 'match_tag']][apply_conditional_iam.user_and_email == user_]) > 1:
        condition_expression.append(' || '.join(set(list(apply_conditional_iam[['user_and_email', 'match_tag']][apply_conditional_iam.user_and_email == user_].match_tag.values))))
    else:
        condition_expression.append(apply_conditional_iam[['user_and_email', 'match_tag']][apply_conditional_iam.user_and_email == user_].match_tag.values[0])

user_condition_expression = pd.DataFrame(data = {'user_and_email': apply_conditional_iam.user_and_email.unique(), 'condition_expression': condition_expression})

In [ ]:
role_with_condition_members = []

for condition_expression_ in user_condition_expression.condition_expression.unique():

    role_with_condition_members.append(list(user_condition_expression[user_condition_expression.condition_expression == condition_expression_].user_and_email.values))

In [ ]:
condition_expressions_and_members_list = pd.DataFrame(data = {'members_list': role_with_condition_members, 'condition_expression':user_condition_expression.condition_expression.unique()})

In [ ]:
condition_expressions_and_members_list['title'] = ['Condition ' + str(index + 1) for index in range(len(condition_expressions_and_members_list))]

In [ ]:
#Tratamento da role dataViewer geral
for binding in json_data['bindings']:
    if binding['role'] == 'roles/bigquery.dataViewer' and 'condition' not in binding:
        general_binding = binding

for e_ in user_condition_expression.user_and_email.values:
    try:
        general_binding['members'].remove(e_)
    except:
        pass

In [ ]:
conditional_binding = []

for index in range(len(condition_expressions_and_members_list)):

    new_binding_ = {
        'role': 'roles/bigquery.dataViewer',
        'members': condition_expressions_and_members_list.members_list.values[index],
        'condition': {'expression': condition_expressions_and_members_list.condition_expression[index], 'title': condition_expressions_and_members_list.title[index]}
    }

    conditional_binding.append(new_binding_)


In [ ]:
json_data['bindings'] = [binding for binding in json_data['bindings'] if binding['role'] != 'roles/bigquery.dataViewer']

In [ ]:
json_data['bindings'].append(general_binding)

In [ ]:
for new_binding_ in conditional_binding:

    json_data['bindings'].append(new_binding_)

In [ ]:
for binding in json_data['bindings']:
    if binding['role'] == 'roles/bigquery.dataViewer':
        print(binding, '\n')

In [ ]:
reordered = {
    "etag": json_data["etag"],
    "version": json_data["version"],
    "bindings": json_data["bindings"]
}

# Convert dictionary to JSON string
json_string = json.dumps(reordered)
print(json_string)

In [ ]:
### ALTERAR POLICIES

# Prepare the URL for the GET request to list all tag keys
url = f'https://cloudresourcemanager.googleapis.com/v1/projects/{PROJECT_ID}:setIamPolicy'

# Prepare the headers with the obtained access token
headers = {
    'Authorization': f"Bearer {credentials__.token}",
    'Content-Type': 'application/json'
}

# POST request data
data = '{ \"policy\":' + json_string + '}'

# Make the API request
response = requests.post(url, headers=headers, data=data)

if response.status_code == 200:
    print("Sucess!", response.status_code, response.text)
elif response.status_code == 403:
    print("Failed:", response.status_code, response.text)
elif response.status_code == 404:
    print("Failed:", response.status_code, response.text)
else:
    print("Failed:", response.status_code, response.text)

In [ ]:
condition_expressions_and_members_list ##generate version control in BigQuery